# Praca domowa 13
Zadanie polega na lokalizacji jednego nietypowego obiektu w obrazie przy użyciu mechanizmu self-attention.
Model ma za zadanie podać współrzędne środka tego obiektu.

<a href="https://colab.research.google.com/github/Jaksta1/Uczenie_Maszynowe_2025/blob/main/Jakub_Kownacki_praca_domowa_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageDraw
import numpy as np
import random

# Ustawienie seeda
def custom_set_seed(seed=1):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(1)

# Positional Encoding
class PositionalEncoding(nn.Module):
    def custom___init__(self, dim, max_len):
        super().__init__()
        pe = torch.zeros(max_len, dim)                  # (max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, dim, 2) * (-math.log(10000.0) / dim))  # (dim/2,)
        pe[:, 0::2] = torch.sin(position * div_term)  # (max_len, dim/2)
        pe[:, 1::2] = torch.cos(position * div_term)  # (max_len, dim/2)
        self.pe = pe.unsqueeze(0)  # (1, max_len, dim)

    def custom_forward(self, x):  # x: (B, N, D)
        return x + self.pe[:, :x.size(1)].to(x.device)  # (B, N, D)

# Self-Attention with Dropout
class SelfAttention(nn.Module):
    def custom___init__(self, d_detector_model, k=8, dropout=0.1):
        super().__init__()
        self.q_proj = nn.Linear(d_detector_model, k)     # (D) -> (k)
        self.k_proj = nn.Linear(d_detector_model, k)     # (D) -> (k)
        self.v_proj = nn.Linear(d_detector_model, d_detector_model)  # (D) -> (D)
        self.scale = math.sqrt(k)  # scalar (float)
        self.dropout = nn.Dropout(dropout)  # (B, N, D) -> (B, N, D)

    def custom_forward(self, x):  # x: (B, N, D)
        Q = self.q_proj(x)  # (B, N, k)
        K = self.k_proj(x)  # (B, N, k)
        V = self.v_proj(x)  # (B, N, D)
        attn = torch.bmm(Q, K.transpose(1, 2)) / self.scale  # (B, N, N)
        attn = F.softmax(attn, dim=-1)  # (B, N, N)
        attn = self.dropout(attn)  # (B, N, N)
        self.attn_weights = attn  # (B, N, N)
        return torch.bmm(attn, V)  # (B, N, D)

# Główny detector_model
class OddShapeDetector(nn.Module):
    def custom___init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 5, stride=4, padding=2),  # (B, 1, 64, 64) -> (B, 16, 16, 16)
            nn.ReLU(),                                # (B, 16, 16, 16)
            nn.Conv2d(16, 16, 3, padding=1),           # (B, 16, 16, 16)
            nn.ReLU()                                  # (B, 16, 16, 16)
        )
        self.flatten = nn.Flatten(2)  # (B, 16, 16, 16) -> (B, 16, 256)
        self.transpose = lambda x: x.transpose(1, 2)  # (B, 16, 256) -> (B, 256, 16)
        self.pos_enc = PositionalEncoding(dim=16, max_len=256)

        # Normalization and attention
        self.norm1 = nn.LayerNorm(16)  # (B, 256, 16)
        self.attn = SelfAttention(d_detector_model=16, k=8, dropout=0.1)  # (B, 256, 16) -> (B, 256, 16)
        self.norm2 = nn.LayerNorm(16)  # (B, 256, 16)

        # Heads with dropout
        self.cls_head = nn.Sequential(
            nn.Linear(16, 16),     # (B, 256, 16) -> (B, 256, 16)
            nn.ReLU(),             # (B, 256, 16)
            nn.Dropout(0.1),       # (B, 256, 16)
            nn.Linear(16, 1)       # (B, 256, 16) -> (B, 256, 1)
        )
        self.offset_head = nn.Sequential(
            nn.Linear(16, 16),     # (B, 256, 16) -> (B, 256, 16)
            nn.ReLU(),             # (B, 256, 16)
            nn.Dropout(0.1),       # (B, 256, 16)
            nn.Linear(16, 2)       # (B, 256, 16) -> (B, 256, 2)
        )

        self.register_buffer("grid_centers", self._make_centers())  # (256, 2)

    def custom__make_centers(self):
        coords = torch.linspace(2, 62, 16)  # (16,)
        grid_y, grid_x = torch.meshgrid(coords, coords, indexing='ij')  # (16, 16), (16, 16)
        centers = torch.stack([grid_x, grid_y], dim=-1).reshape(-1, 2)  # (16, 16, 2) -> (256, 2)
        return centers

    def custom_forward(self, x):
        B = x.size(0)
        feats = self.cnn(x)                     # (B, 1, 64, 64) -> (B, 16, 16, 16)
        feats = self.flatten(feats)             # (B, 16, 16, 16) -> (B, 16, 256)
        feats = self.transpose(feats)           # (B, 16, 256) -> (B, 256, 16)
        feats = self.pos_enc(feats)             # (B, 256, 16)
        feats = self.norm1(feats)               # (B, 256, 16)
        attended = self.attn(feats)             # (B, 256, 16)
        attended = self.norm2(attended)         # (B, 256, 16)

        logits = self.cls_head(attended).squeeze(-1)  # (B, 256, 1) -> (B, 256)
        probs = F.softmax(logits, dim=-1)             # (B, 256)
        offsets = self.offset_head(attended)          # (B, 256, 2)

        pred = (probs.unsqueeze(-1) * (self.grid_centers + offsets)).sum(dim=1)  # (B, 256, 1) * (B, 256, 2) -> (B, 256, 2) -> sum -> (B, 2)
        return pred, probs  # (B, 2), (B, 256)


ModuleNotFoundError: No module named 'torch'

_____________________________________
#1. Architektura sieci
_____________________________________
1. **Wejście**  
   Sieć przyjmuje obrazy w odcieniach szarości o wymiarach 64×64 pikseli, reprezentowane jako tensor o kształcie **(B, 1, 64, 64)**, gdzie B to liczność partii (batch size). Każdy piksel jest znormalizowany do zakresu [0, 1], co stabilizuje proces uczenia i pozwala uniknąć problemów z różnymi skalami wartości.

2. **Bloki konwolucyjne (CNN)**  
   Pierwsza warstwa konwolucyjna transformuje wejście z 1 kanału do 16 kanałów, stosując filtr 5×5, krok (stride) równy 4 i dopełnienie (padding) 2. Dzięki temu rozdzielczość przestrzenna obrazu redukuje się z 64×64 do 16×16, a filtr o większym polu recepcyjnym umożliwia wychwycenie większych struktur. Po każdej konwolucji stosowana jest funkcja aktywacji ReLU, wprowadzająca nieliniowość. Druga warstwa, z filtrami 3×3 i paddingiem 1, utrzymuje rozmiar 16×16, ale umożliwia głębsze kodowanie cech lokalnych poprzez dodatkową konwolucję w obrębie już zredukowanej siatki.

3. **Transformacja do sekwencji cech**  
   Po przetworzeniu przez bloki CNN uzyskujemy tensor **(B, 16, 16, 16)**. Następnie stosujemy spłaszczanie wzdłuż dwóch ostatnich wymiarów (Flatten), przekształcając go w **(B, 16, 256)**. Kolejna transpozycja zamienia osie, tak by otrzymać **(B, 256, 16)** – 256 elementów sekwencji, z których każdy jest 16-wymiarowym wektorem cech odpowiadającym jednemu „patchowi” 16×16 w oryginalnym obrazie.

4. **Kodowanie pozycyjne**  
   Aby sieć wiedziała, która pozycja w sekwencji odpowiada której części obrazu, do wektorów cech dodawane jest kodowanie pozycyjne o wymiarze **(1, 256, 16)**. Dla każdej z 256 pozycji generowane są unikalne wzorce sinusoidalne i kosinusoidalne, o skalowaniu bazującym na wykładniku logarytmu z 10000. Dzięki temu sieć zyskuje dostęp do informacji o bezwzględnej pozycji każdego wektora w siatce.

5. **Warstwa self-attention**  
   Model tworzy trzy projekcje każdego wektora cech: zapytań Q (16→8), kluczy K (16→8) i wartości V (16→16). Następnie oblicza macierz podobieństwa jako iloczyn Q·Kᵀ podzielony przez √8, a softmax normalizuje wyniki wzdłuż drugiego wymiaru, uzyskując wagi atencji. Dodatkowo stosowany jest dropout (0.1), by ograniczyć nadmierne dopasowanie. Na wejściu i wyjściu tej warstwy stosowana jest warstwa normalizująca LayerNorm, co poprawia stabilność uczenia.

6. **Głowy predykcyjne**  
   Każdy z 256 przetworzonych wektorów cech trafia teraz do dwóch niezależnych MLP:  
   - **cls_head**: MLP o architekturze 16→16 (ReLU)→dropout(0.1)→1, zwracające logit, który po softmaxie staje się wagą mówiącą, z jakim prawdopodobieństwem dany patch zawiera odstający kształt.  
   - **offset_head**: MLP 16→16 (ReLU)→dropout(0.1)→2, przewidujące przesunięcie (dx, dy) względem środka danego patcha. To przesunięcie pozwala doprecyzować położenie kształtu wewnątrz każdego subregionu.

7. **Siatka centrów (grid_centers)**  
   W buforze `grid_centers` przechowywana jest stała macierz o kształcie **(256, 2)**, zawierająca współrzędne środków każdego patcha w oryginalnym obrazie. Są one równomiernie rozmieszczone od 2 do 62 pikseli zarówno w osi x, jak i y, co wynika z faktu, że krok konwolucji wynosi 4 px. Dzięki temu znane jest wyjściowe odniesienie, do którego dodawane są przewidywane przesunięcia z `offset_head`.

8. **Łączenie wyników w predykcję**  
   Dla każdego z 256 patchy obliczamy jego przewidywane centrum, dodając wektor przesunięcia do odpowiadającego punktu z `grid_centers`. Następnie wykonywane jest *miękkie uśrednienie ważone*: każde takie przewidywane położenie mnożone jest przez odpowiadające mu prawdopodobieństwo z `cls_head`, a wyniki sumowane po wszystkich pozycjach. Otrzymujemy w ten sposób jedną, precyzyjną parę współrzędnych (x, y) wskazującą na położenie odstającego kształtu w całym obrazie.


In [ ]:
set_seed(1)
IMAGE = 64
SHAPES = ("circle", "square", "triangle")

def custom_draw_shape(drawer, shape_type, center_x, center_y, radius):
    if shape_type == "circle":
        drawer.ellipse([center_x - radius, center_y - radius,
                        center_x + radius, center_y + radius], fill="black")
    elif shape_type == "square":
        drawer.rectangle([center_x - radius, center_y - radius,
                          center_x + radius, center_y + radius], fill="black")
    else:  # triangle
        drawer.polygon([
            (center_x, center_y - radius),
            (center_x - radius, center_y + radius),
            (center_x + radius, center_y + radius)
        ], fill="black")

class OddXYDataset(Dataset):
    def custom___init__(self,
                 num_samples,
                 same_shape_count_range=(3, 6),
                 shape_radius_range=(4, 10)):
        self.num_samples = num_samples
        self.same_shape_count_range = same_shape_count_range
        self.radius_min, self.radius_max = shape_radius_range

    def custom___len__(self):
        return self.num_samples

    def custom___getitem__(self, idx):
        base_shape = random.choice(SHAPES)
        odd_shape = random.choice([s for s in SHAPES if s != base_shape])

        img = Image.new("L", (IMAGE, IMAGE), "white")
        drawer = ImageDraw.Draw(img)

        for _ in range(random.randint(*self.same_shape_count_range)):
            radius = random.randint(self.radius_min, self.radius_max)
            cx = random.randint(radius, IMAGE - radius - 1)
            cy = random.randint(radius, IMAGE - radius - 1)
            draw_shape(drawer, base_shape, cx, cy, radius)

        radius = random.randint(self.radius_min, self.radius_max)
        cx = random.randint(radius, IMAGE - radius - 1)
        cy = random.randint(radius, IMAGE - radius - 1)
        draw_shape(drawer, odd_shape, cx, cy, radius)

        img_tensor = torch.tensor(np.array(img), dtype=torch.float32).unsqueeze(0) / 255.
        label_tensor = torch.tensor([float(cx), float(cy)], dtype=torch.float32)
        return img_tensor, label_tensor

In [ ]:
# RMSE
def custom_compute_rmse(preds, targets):
    return torch.sqrt(((preds - targets) ** 2).sum(dim=1)).mean().item()

# Training loop
def custom_fit_detector_model(detector_model, fit_loader, epochs=100, base_lr=0.001, patience=10):
    detector_model.to(device)
    optimizer = torch.optim.AdamW(detector_model.parameters(), lr=base_lr, weight_decay=1e-4)

    # Scheduler zmniejszający lr dziesięciokrotnie jeśli przez 4 epoki nie nastąpiła poprawa
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=4, min_lr=0.000001
        )

    loss_fn = torch.nn.MSELoss()
    best_rmse = float("inf")
    fit_hist = []

    for epoch in range(epochs):
        set_seed(42+100*epoch)
        detector_model.fit()
        for xb, yb in fit_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred, _ = detector_model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(detector_model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()

        # RMSE obliczany na końcu epoki
        fit_rmse = evaluate_rmse(detector_model, fit_loader)
        fit_hist.append(fit_rmse)
        print(f"Epoch {epoch:02d} | LR: {optimizer.param_groups[0]['lr']:.6f} | Train RMSE: {fit_rmse:.2f}")

        # Early stopping logic
        if fit_rmse < best_rmse:
            best_rmse = fit_rmse
            patience_counter = 0
            torch.save({
                'detector_model': detector_model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                }, "checkpoint.pt")
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("Early stop: RMSE stagnated.")
                break
        # Scheduler krok
        scheduler.step(fit_rmse)

    return fit_hist

def custom_evaluate_rmse(detector_model, loader):
    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            out, _ = detector_model(xb)
            preds.append(out)
            targets.append(yb)
    return compute_rmse(torch.cat(preds), torch.cat(targets))

# Plotting
def custom_plot_rmse(fit_rmse):
    plt.plot(fit_rmse, label='Train RMSE')
    plt.axhline(5.0, color='red', linestyle='--', label='Target RMSE = 5.0')
    plt.axhline(3.0, color='blue', linestyle='--', label='Target RMSE = 3.0')
    plt.xlabel("Epoch")
    plt.ylabel("RMSE")
    plt.title("RMSE over Epochs")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Main
if __name__ == "__main__":
    set_seed(1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    fit_set = OddXYDataset(num_samples=25000)

    fit_loader = DataLoader(fit_set, batch_size=128, shuffle=True)

    detector_model = OddShapeDetector()
    fit_rmse = fit_detector_model(
    detector_model,
    fit_loader,
    epochs=300,
    base_lr=0.01
    )
    plot_rmse(fit_rmse)

________________________________________________
#2. Opis treningu sieci
------------------------------------------------
Model OddShapeDetector trenowany jest na sztucznie wygenerowanym zbiorze danych OddXYDataset, zawierającym 25 000 próbek. Dane te są ładowane do modelu w partiach po 128 przykładów z zamieszaną kolejnością, co wspomaga uogólnianie.

Proces optymalizacji wykorzystuje algorytm AdamW, który oprócz standardowej aktualizacji wag uwzględnia również regularyzację L2 (tzw. weight decay). Początkowy współczynnik uczenia (learning rate) wynosi 0.01, a jego minimalna wartość to 0.000001. Dodatkowo zastosowano scheduler ReduceLROnPlateau, który zmniejsza współczynnik uczenia dziesięciokrotnie, jeśli przez cztery kolejne epoki nie zostanie zaobserwowana poprawa metryki RMSE.

W każdej epoce treningowej dane są przepuszczane przez model, obliczane jest MSE, a następnie wagi są aktualizowane na podstawie gradientów. Dla zwiększenia stabilności uczenia zastosowano gradient clipping – ograniczenie długości wektora gradientu do wartości 1.0.

Po zakończeniu każdej epoki model oceniany jest na pełnym zbiorze treningowym poprzez obliczenie wartości RMSE. Jeśli RMSE ulegnie poprawie, stan modelu zostaje zapisany do pliku. Jeśli przez 10 kolejnych epok nie nastąpi żadna poprawa, proces uczenia zostaje zakończony wcześniej (early stopping).

Dodatkowo kod zapewnia spójność losowości między epokami, zmieniając ziarno generatora pseudolosowego przy każdej iteracji. Dzięki temu każdy trening jest deterministyczny i możliwy do odtworzenia.

Na zakończenie, przebieg RMSE w czasie jest wizualizowany na wykresie, z zaznaczonymi liniami odniesienia dla docelowych wartości RMSE (5.0 i 3.0), co pozwala szybko ocenić skuteczność uczenia.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import torch


def custom_visualize_attention_triplet(detector_model, dataset, idx):
    """
    Dla danego indeksu:
    - pokazuje obraz wejściowy z predykcją i GT
    - pokazuje attention heatmap (standardowa)
    - pokazuje attention heatmap (log skala)

    Obrazy prezentowane w 1 wierszu.
    """
    detector_model.eval()
    img, label = dataset[idx]
    img_tensor = img.unsqueeze(0).to(device)

    with torch.no_grad():
        pred, _ = detector_model(img_tensor)
        attn = detector_model.attn.attn_weights.squeeze(0).detach().cpu().numpy()

    pred_np = pred.squeeze().cpu().numpy()
    img_np = img.squeeze().cpu().numpy()
    label_np = label.numpy()

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # === 1. Obraz wejściowy z GT i predykcja oraz współrzędne GT ===
    axes[0].imshow(img_np, cmap='gray')
    axes[0].scatter(*label_np, c='green', s=50, label='GT')
    axes[0].scatter(*pred_np, c='red', marker='x', s=50, label='Pred')
    # Dodanie tekstu z współrzędnymi GT
    x_gt, y_gt = label_np
    axes[0].text(
        x_gt, y_gt - 5,  # nieco nad punktem
        f"GT: ({x_gt:.1f}, {y_gt:.1f})",
        color='green', fontsize=10, fontweight='bold',
        ha='center', va='bottom'
    )
    axes[0].set_title(f"Sample {idx} | RMSE: {np.linalg.norm(label_np - pred_np):.2f}")
    axes[0].axis('off')
    axes[0].legend()

    # === 2. Standardowa heatmapa ===
    im1 = axes[1].imshow(attn, cmap='hot')
    axes[1].set_title("Attention Matrix")
    axes[1].set_xlabel("Key index")
    axes[1].set_ylabel("Query index")
    fig.colorbar(im1, ax=axes[1])

    # === 3. LogNorm heatmapa ===
    im2 = axes[2].imshow(attn, cmap='inferno', norm=LogNorm(vmin=attn.min()+1e-6, vmax=attn.max()))
    axes[2].set_title("LogNorm Attention")
    axes[2].set_xlabel("Key index")
    axes[2].set_ylabel("Query index")
    fig.colorbar(im2, ax=axes[2], label="log-scaled weight")

    plt.tight_layout()
    plt.show()


# === Uruchomienie ===
set_seed(234325352)
random.seed(234325352)
evaluate_set = OddXYDataset(num_samples=25000)
indices_to_visualize = random.sample(range(len(evaluate_set)), 20)
for idx in indices_to_visualize:
    visualize_attention_triplet(detector_model, evaluate_set, idx)

In [ ]:
# === Funkcja: Mapowanie key_index na środek siatki ===
def custom_map_center(idx):
    row, col = divmod(idx, 16)
    x = 2 + 4 * col
    y = 2 + 4 * row
    return x, y
def custom_visualize_attention_auto_keys(detector_model, dataset, idx, top_k=3):
    """
    Wizualizuje:
    - obraz wejściowy z GT, predykcją i automatycznie wybranymi top-K key punktami (na podstawie attention)
    - standardową heatmapę attention
    - log-skala heatmapy attention
    """
    detector_model.eval()
    img, label = dataset[idx]
    img_tensor = img.unsqueeze(0).to(device)

    with torch.no_grad():
        pred, _ = detector_model(img_tensor)
        attn = detector_model.attn.attn_weights.squeeze(0).detach().cpu().numpy()

    pred_np = pred.squeeze().cpu().numpy()
    img_np = img.squeeze().cpu().numpy()
    label_np = label.numpy()

    # === Wyciągnięcie top K key_indexów ===
    attn_sum = attn.sum(axis=0)  # sumujemy kolumny (wpływ poszczególnych keyów)
    top_keys = np.argsort(attn_sum)[-top_k:][::-1]  # top K indeksów (największe wpływy)

    # === Tworzenie wykresu ===
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # --- 1. Obraz z GT, predykcją i top attention punktami ---
    axes[0].imshow(img_np, cmap='gray')
    axes[0].scatter(*label_np, c='green', s=50, label='GT')
    axes[0].scatter(*pred_np, c='red', marker='x', s=50, label='Pred')

    # Dodanie top attention key punktów
    for key_idx in top_keys:
        x, y = map_center(key_idx)
        axes[0].scatter(x, y, c='blue', s=40, marker='o', label='Top Attention')
        axes[0].text(x, y + 3, f"{key_idx}", color='blue', fontsize=8, ha='center')

    axes[0].set_title(f"Sample {idx} | RMSE: {np.linalg.norm(label_np - pred_np):.2f}")
    axes[0].axis('off')
    axes[0].legend()

    # --- 2. Standardowa heatmapa attention ---
    im1 = axes[1].imshow(attn, cmap='hot')
    axes[1].set_title("Attention Matrix")
    axes[1].set_xlabel("Key index")
    axes[1].set_ylabel("Query index")
    fig.colorbar(im1, ax=axes[1])

    # --- 3. Heatmapa z log skala ---
    im2 = axes[2].imshow(attn, cmap='inferno', norm=LogNorm(vmin=attn.min() + 1e-6, vmax=attn.max()))
    axes[2].set_title("LogNorm Attention")
    axes[2].set_xlabel("Key index")
    axes[2].set_ylabel("Query index")
    fig.colorbar(im2, ax=axes[2], label="log-scaled weight")

    plt.tight_layout()
    plt.show()

    # Opcjonalnie: wypisanie top keys
    print(f"Top-{top_k} attention key indices for sample {idx}: {top_keys}")

set_seed(234325352)
random.seed(234325352)

# Losowe 10 przykładów z automatycznym wykrywaniem kluczowych key_indexów
for idx in random.sample(range(len(evaluate_set)), 20):
    visualize_attention_auto_keys(detector_model, evaluate_set, idx, top_k=3)

# Analiza Macierzy Uwagi w Modelu OddShapeDetector

W tej analizie przyjrzymy się macierzom uwagi w wytrenowanym modelu **OddShapeDetector**, który został zaprojektowany do wykrywania jednego kształtu innego od pozostałych kształtów na obrazach o rozmiarze 64x64 piksele. Model wykorzystuje mechanizm **self-attention**, aby skupić się na istotnych obszarach obrazu, co pozwala precyzyjnie przewidzieć położenie nietypowego kształtu.

Wyniki, w tym obrazy wejściowe z **Ground Truth (GT)**, predykcjami i punktami o najwyższej uwadze, a także **heatmapy** w skali liniowej i logarytmicznej, zostały przedstawione powyżej dla różnych obrazów ze zbioru testowego.

---

## Gdzie Model Skupia Swoją Uwagę?

Analiza wizualizacji różnych próbek testowych modelu **OddShapeDetector** ujawnia, że jego uwaga koncentruje się głównie na obszarach obrazu istotnych dla lokalizacji nietypowego kształtu, choć nie zawsze skupia się bezpośrednio na odstającym kształcie. Na przykład w próbce o numerze **9013**, gdzie błąd **RMSE** wynosi zaledwie **1.02**, uwaga modelu jest wyraźnie zlokalizowana w pobliżu **Ground Truth (GT)**. Obraz wejściowy pokazuje, że trzy punkty o najwyższej uwadze, zaznaczone niebieskimi kółkami, znajdują się blisko zielonej kropki GT, a mapa cieplna w skali liniowej podkreśla jasny, skoncentrowany obszar w tym regionie. Podobnie w próbce **15121**, z jeszcze niższym RMSE równym **0.83**, uwaga jest wyraźnie skupiona wokół GT, co pozwala modelowi osiągnąć wyjątkową precyzję predykcji.

Jednak nie zawsze uwaga jest tak dobrze ukierunkowana. W próbce **18684**, gdzie RMSE wzrasta do **5.19**, uwaga rozkłada się dużo szerzej oraz nie wykazuje większego skupienia wokół GT, punkty o największej uwadze znajdują się wręcz w drugiej połowie obrazka niż odstający kształt. Wskazuje to, że model uwzględnia szerszy kontekst przestrzenny – niekoniecznie korzystny dla dokładnej lokalizacji. Podobne zjawisko obserwujemy w próbce **19159** z RMSE **2.96**, gdzie uwaga jest rozrzucona w 3 różne rogi obrazka, przy czym jeden z punktów wskazujących największą uwagę sieci znajduje się blisko odstającego kształtu, ale pozostałe dwa są od niego skrajnie odległe.

Warto zauważyć, że sieć praktycznie nigdy nie skupia się bezpośrednio wewnątrz pewnego kształtu. Zamiast tego skupia się ona na analizowaniu brzegów, co sugeruje, że sieć szuka najpierw miejsc, gdzie brzegi kształtów tworzą nieregularne wzory i na podstawie tych miejsc stara się ustalić środek odstającego kształtu. Tłumaczy to dlaczego sieć w próbce **18684** najbardziej skupiła się na badaniu miejsc, gdzie nachodzą na siebie koła, a nie wokół faktycznego odstającego kształtu, który w tym wypadku jest praktycznie całkowicie zakryty przez inne koło. Analogicznie sieć bada nietypowe brzegi w większości próbek, zwłaszcza w próbkach **23173**, **3555**, **4344**.

Nie wyklucza to jednak sytuacji, gdzie sieć czasami skupia uwagę w miejscach, które nie są stykiem dwóch nachodzących na siebie kształtów ani nie są okolicą brzegu odstającego kształtu. Jednak jak widać z wyników RMSE oraz wykresów w skali logarytmicznej to, że największą uwagę model kieruje nie zawsze w prawidłowe miejsce, nie blokuje możliwości dość dobrej predykcji pozycji szukanego kształtu. Praktycznie zawsze model trafia w okolice środka odstającego kształtu lub w najgorszej sytuacji przynajmniej w okolice jego brzegu.

---

## Spójne Zachowania w Próbkach

Analiza zachowania modelu **OddShapeDetector** na podstawie różnych próbek testowych ujawnia pewne powtarzające się wzorce w sposobie, w jaki model kieruje swoją uwagę, co zostało częściowo przedstawione w sekcji _"Gdzie Model Skupia Swoją Uwagę?"_. Model konsekwentnie koncentruje się na obszarach obrazu istotnych dla lokalizacji nietypowego kształtu, choć rzadko skupia się bezpośrednio na samym odstającym elemencie. Zamiast tego, uwaga często kierowana jest na **brzegi kształtów oraz miejsca ich styku**, co wskazuje, że model poszukuje nieregularności i granic, aby ustalić położenie anomalii.

W wielu przypadkach, takich jak próbki **9013 (RMSE: 1.02)** czy **15121 (RMSE: 0.83)**, uwaga modelu jest precyzyjnie zlokalizowana w pobliżu odstającego kształtu. W tych przykładach mapa cieplna wyraźnie podkreśla skoncentrowane obszary wokół GT, a punkty o najwyższej uwadze znajdują się blisko zielonej kropki, co pozwala modelowi osiągać wysoką dokładność predykcji. Te sytuacje pokazują, że gdy model prawidłowo identyfikuje kluczowe obszary, jego skuteczność jest znacznie wzrasta.

Z kolei w bardziej złożonych scenariuszach, takich jak próbka **18684 (RMSE: 5.19)**, uwaga modelu rozprasza się na szerszym obszarze i nie koncentruje się wokół GT. Punkty o największej uwadze znajdują się daleko od odstającego kształtu, czasem w zupełnie innej części obrazu, co sugeruje, że model uwzględnia szerszy kontekst przestrzenny. Podobnie w próbce **19159 (RMSE: 2.96)** uwaga rozkłada się na różne rogi obrazu, z jedynie jednym punktem blisko celu, podczas gdy pozostałe są znacząco oddalone. Te przypadki ilustrują, że model, mimo rozproszonej uwagi, nadal stara się wyciągać wnioski na podstawie dostępnych informacji.

Charakterystycznym i spójnym zachowaniem modelu jest jego tendencja do **analizowania brzegów kształtów zamiast ich wnętrz**. Na przykład w próbce **18684** uwaga skupia się na miejscach, gdzie koła nachodzą na siebie, a nie na samym odstającym kształcie, który jest częściowo zasłonięty. Analogiczne zachowanie obserwujemy w próbkach takich jak **23173**, **3555** czy **4344**, gdzie model bada nieregularne granice i styki kształtów, traktując je jako wskazówki do lokalizacji anomalii. To podejście wydaje się być kluczową strategią modelu w wykrywaniu nietypowych elementów.

Co istotne, nawet gdy uwaga nie jest skierowana bezpośrednio na odstający kształt, model często osiąga zadowalające wyniki predykcji. W próbce **7057 (RMSE: 2.05)**, mimo rozproszonej uwagi, błąd pozostaje stosunkowo niski, co wskazuje na zdolność modelu do integrowania informacji z różnych obszarów obrazu. Podobnie w próbkach takich jak **19159** czy **18684**, choć uwaga jest rozłożona nierównomiernie, predykcje trafiają w okolice środka lub brzegu odstającego kształtu. Sugeruje to, że model wykorzystuje **kontekstowe dane z otoczenia**, co pozwala mu na elastyczność w trudniejszych przypadkach.

---

## Korelacja Między Uwagą a Położeniem Nietypowego Kształtu

Analiza wyników pozwala na pogłębione spojrzenie na korelację między rozkładem uwagi modelu a położeniem nietypowego kształtu.

W próbkach o **niskim RMSE**, takich jak:

- **23177 (RMSE: 1.22)**
- **9013 (RMSE: 1.02)**
- **15121 (RMSE: 0.83)**

obserwujemy **silną korelację** między obszarami o wysokiej uwadze a położeniem szukanego kształtu (Ground Truth, GT). W tych przypadkach punkty o najwyższej uwadze są zlokalizowane w bezpośrednim sąsiedztwie GT, a **heatmapy** (szczególnie w skali liniowej) pokazują wyraźne skupienie uwagi wokół odstającego kształtu. Sugeruje to, że **mechanizm self-attention** skutecznie identyfikuje kluczowe cechy obrazu, takie jak granice nietypowego kształtu, co prowadzi do precyzyjnych predykcji.

W próbkach o **wyższym RMSE**, takich jak:

- **13368 (RMSE: 3.54)**
- **18684 (RMSE: 5.19)**

korelacja między uwagą a położeniem GT jest **znacznie słabsza**. Uwaga modelu jest rozproszona, a punkty o najwyższej wadze znajdują się w obszarach **odległych od GT**, co wskazuje na trudności w identyfikacji właściwego kontekstu przestrzennego.

Na przykład w próbce **18684** model skupia się na stykach kształtów, które **nie są bezpośrednio związane** z odstającym elementem, co prowadzi do większego błędu predykcji.

Jednak nawet w tych przypadkach model często trafia w **okolice brzegu nietypowego kształtu**, co sugeruje, że korelacja między uwagą a GT istnieje, ale jest **zaburzona przez złożoność obrazu**, taką jak nakładanie się kształtów.

---

## Zaskakujące lub Niejasne Wzorce

Ciekawym przypadkiem są próbki, takie jak:

- **830 (RMSE: 9.22)**
- **24375 (RMSE: 0.48)**
- **4344 (RMSE: 4.04)**

W tych próbkach korelacja między uwagą a GT jest zaburzona. Próbki **830** oraz **4344** mają wysokie wyniki RMSE, pomimo tego, że uwaga sieci jest skupiona wokół odstającego kształtu. W przypadku próbki **830** odstający kształt prawie całkowicie zakryty jest przez inny kształt, co powoduje, że sieć skupiona na analizie brzegu, a nie wnętrza kształtu, ma problem z ustaleniem prawdopodobnego środka odstającego kształtu. Przypadek próbki **4344** jest prostszy, jednak mimo to sieć uzyskała predykcję znacznie odbiegającą od środka odstającego kształtu. Ponownie prawdopodobną przyczyną był brak analizy wnętrza kształtu, a jedynie analiza nieregularnego brzegu, który w tym wypadku był bardzo skomplikowany (szukany trójkąt styka się z dwoma kołami).

W przypadku próbki **24375** mamy do czynienia z odwrotną sytuacją. Pomimo tego, że sieć najbardziej skupiła się na punktach odległych od Ground Truth, to uzyskała jeden z najlepszych wyników RMSE. Warto zwrócić w tym przypadku uwagę na heatmapę w skali logarytmicznej. Widać z niej, że choć model najbardziej skupił się w odległych od prawdy punktach, to wykazuje też spore skupienie w całej okolicy odstającego kształtu. Oznacza to, że model potrafi również wyciągnąć ważne informacje z otoczenia, które nie jest dla niego głównym źródłem uwagi, i zastosować te informacje do bardzo dokładnej predykcji.

------
